# Episode Event to OMOP


Episode event:
https://docs.google.com/spreadsheets/d/1Vw1Dr2K4oG__cDQTutGaJhZvGUvQTLwc4qWreP6qMSs/edit?gid=857970120#gid=857970120

Episodes represent the evolution of the cancer during time. For example:
![3rd image](images/EpisodeEventIDEA3.png)

In OMOP we have 2 kind of episode, dynamic that are the temporal evolution of the cancer and the extent that is the spatial evolution of the cancer.

The episode event in idea4rc models the dynamic of the disease

![First image](images/EpisodeEventIDEA1.png)



In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta
from uuid import uuid4

In [ ]:
df_Episode_Event_IDEA4RC = pd.read_csv("./IDEA4RC-data/episodeEventIDEA4RC.csv")
df_Episode_Event_IDEA4RC.head(5)

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### Doubts

Why we need is new cancer variable?
In every case we need a new entry in cancerepisode table to construct the new cancer diagonosis, once we have a new cancerepisode all related events will be linked to this other overarching episode

Our concept of wrong diagnosis is represented in omop?

![First image](images/EpisodeEventIDEA1.png)

In IDEA4RC the disease extent should be in the Stages

![Second image](images/EpisodeEventIDEA2.png)

We need to see if the datamodel changes for New diagnosis and Is Diagnosis Wrong. But we are still keeping the mapping to the measurements (maybe these should be observations instead?)

### Episode Event to Episode

This Episode will be a child episode, so we can track the cancer's progression in OMOP.

episode_object_concept_id???

In [ ]:
sql = """
    INSERT INTO omopcdm.episode (person_id, episode_start_date, episode_concept_id, episode_object_concept_id, episode_type_concept_id, episode_parent_id)
    VALUES (%s, %s, %s, %s, %s, %s)
"""
query="""
    SELECT person_id
    FROM cancer_episode
    WHERE id = %s
"""
queryDate="""
    SELECT episode_start_date
    FROM cancer_episode
    WHERE id = %s
"""
sqlGetEpisode = """
    SELECT c.id, c.episode_object_concept_id 
    FROM episode c
    WHERE c.person_id = %s
    AND c.episode_start_date = %s
    LIMIT 1
"""
#Episode concept id=vocabulary from event type
event_type_concept_id=32879      #Is this ok?
df_tables=df_Episode_Event_IDEA4RC
for idx, row in df_tables.iterrows():
    curIDEA.execute(query, (row['Cancer Episode Reference']))
    res=curIDEA.fetchone()
    person_id=res
    episode_concept_id=row['Disease status']
    episode_start_date=row['Date of episode']
    curIDEA.execute(queryDate, (row['Cancer Episode Reference']))
    res=curIDEA.fetchone()
    dateAux=res
    cur.execute(sqlGetEpisode, (person_id,dateAux))
    episode_parent_id=cur.fetchone()
    episode_object_concept_id=episode_parent_id[1]
    episode_parent_id=episode_parent_id[0]

    cur.execute(sql, (person_id, episode_start_date, episode_concept_id, episode_object_concept_id, event_type_concept_id, episode_parent_id))
    cur.commit()
    

### Episode Event to Procedure Ocurrance

Using 0 as the value for those columns we do not understand how to manage

In [ ]:

procedure_concept_id=0
procedure_type_concept_id=32879      #This one should be the reason why the diagnosis changed or how they discovered it, I believe, yet we don't have that information in IDEA4RC
sql = """
    INSERT INTO omopcdm.procedure_occurrence (procedure_id, person_id, procedure_concept_id, procedure_date,procedure_type_concept_id,provider_id)
    VALUES (%s, %s, %s, %s, %s,%s)
"""

def episode_event_definition(idEpisode, idProcedure):
    sql = """
        INSERT INTO omopcdm.episode_event (episode_id, event_id, episode_event_field_concept_id )
        VALUES (%s, %s, %s)
    """
    cur.execute(sql, (idEpisode, idProcedure,1147129))
    conn.commit()


for idx, row in df_tables.iterrows():
    curIDEA.execute(query, (row['Cancer Episode Reference']))
    res=curIDEA.fetchone()
    person_id=res
    procedure_date=row['Date of episode']
    provider_id=row['Defined At']
    curIDEA.execute(queryDate, (row['Cancer Episode Reference']))
    res=curIDEA.fetchone()
    dateAux=res
    idProcedure=datetime.now().strftime("%Y%m%d%H%M%S")+ str(uuid4())
    cur.execute(sqlGetEpisode, (person_id,dateAux))
    episode_parent_id=cur.fetchone()
    episode_event_definition(episode_parent_id, idProcedure)
    cur.execute(sql, (idProcedure, person_id, procedure_concept_id, procedure_date,procedure_type_concept_id,provider_id))
    conn.commit()

### New or wrong diagnosis

measurement concept id and type concept id???

In [ ]:
sqlMeasurements = """
    INSERT INTO omopcdm.measurement (measurement_id, person_id, measurement_concept_id, measurement_date, measurement_type_concept_id, value_as_concept_id, modifier_of_event_id,modifier_of_field_concept_id,)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""
modifier_of_field_concept_id=1147127
measurement_concept_id=0
measurement_type_concept_id=32879

queryGetOccurrence="""
    SELECT condition_id
    FROM condition_occurrence
    WHERE person_id = %s AND start_date = %s
"""

for idx, row in df_tables.iterrows():
    curIDEA.execute(query, (row['Cancer Episode Reference']))
    res=curIDEA.fetchone()
    person_id=res
    measurement_date=row['Date of episode']
    cur.execute(queryGetOccurrence, (person_id,measurement_date))
    modifier_of_event_id=cur.fetchone()
    value_as_concept_id = 4188539 if row['Is New Diagnosis'] == 4188539 or row['Is New Diagnosis'] else 4188540 #We need the concept id
    cur.execute(sqlMeasurements, (person_id, measurement_date, measurement_concept_id, measurement_type_concept_id, value_as_concept_id, modifier_of_event_id,modifier_of_field_concept_id))
    value_as_concept_id = 4188539 if row['Is Diagnosis Wrong'] == 4188539 or row['Is Diagnosis Wrong'] else 4188540 #We need the concept id
    cur.execute(sqlMeasurements, (person_id, measurement_date, measurement_concept_id, measurement_type_concept_id, value_as_concept_id, modifier_of_event_id,modifier_of_field_concept_id))
